# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

import missingno as msno
from googletrans import Translator
import Levenshtein

import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as pc
import pickle

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import sys
# from IPython.display import display
# import pprint
# import itertools
# import math
# import nltk
# from nltk.corpus import wordnet

'''
custom modules
'''
# function similar to Excel's vlookup
from vlookup import vlookup
# function to view all contents of a dataframe
from view_all import view_all
# function for the project
from section_1 import msno_overview
from section_2 import similar_feature, attribute_cat, str_to_num, ger_to_eng, pv_min_max, \
    pv_meaning_score, pv_verify_null
from section_3 import dataset_summary, view_feature
from section_4 import miss_val_summary

In [ ]:
# pip install googletrans==4.0.0-rc1
# pip install missingno
# pip install pyarrow
# custom modules for convenience are in root folder

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

### 1. Load and overview

#### 1.1 azdias

In [ ]:
# 제출전 주석 해제
# '''
# load and file overview
# '''

# azdias = pd.read_csv('../csv_pickle/Udacity_AZDIAS_052018.csv', sep=';')
# azdias.name = 'azdias'
#
# print (azdias.info())
# azdias.head()

In [ ]:
azdias = pd.read_pickle('../csv_pickle/azdias.pickle')

print (azdias.info())
azdias.head()

In [ ]:
# '''
# missing value overview
# '''

# azdias.name = 'azdias'
# msno_overview(azdias)

#### 1.2 customers

In [ ]:
# 제출전 주석 해제
# '''
# load and file overview
# '''

# customers = pd.read_csv('../csv_pickle/Udacity_CUSTOMERS_052018.csv', sep=';')
# print (customers.info())
# customers.head()

In [ ]:
customers = pd.read_pickle('../csv_pickle/customers.pickle')

print (customers.info())
customers.head()

In [ ]:
# '''
# missing value overview
# '''

# customers.name = 'customers'
# msno_overview(customers)

#### 1.3 info

In [ ]:
'''
load and overview
'''

ref_info = pd.read_excel(
    'DIAS Information Levels - Attributes 2017.xlsx', header=1).iloc[:, 1:]

# fillna of Information level as values are only in the 1st rows
ref_info['Information level'] = ref_info['Information level'].fillna(method = 'ffill')

print (ref_info.info())
ref_info.head()

#### 1.4 attr

In [ ]:
'''
load and overview
'''

ref_attr = pd.read_excel(
    'DIAS Attributes - Values 2017.xlsx', header=1).iloc[:, 1:]
# rename column Value to Score for easier documentation
ref_attr = ref_attr.rename(columns = {'Value' : 'Score'})

print (ref_attr.info())
ref_attr[103:108]

### 2. Baseline understanding

There are 4 files given:
* 2 dataset files - azdias & customers
* 2 reference files - info & attr:  
  - info is for information on dataset features which are in unreadable German acronyms  
  - attr is for information on dataset values which are in numbers and acronyms,  
  and has corresponding meanings  

To establish a baseline, this section aims to identify key factors in the reference files   
that can help in understanding the contents of the dataset.

#### 2.1 feature alignment

As datasets do not have information on what each column feature exactly means,  
values of reference files have to be mapped to the features.
  
feature_desc below is for this needs, and formed with 369 column features from customers. 

After merging information columns, there are 264 features from both customers and info,  
along with 105 exclusive features of customers lacking Description values.

< 2개 내용을 합침  6/7>

<!-- #### 2.2 feature alignment -->

The improved readability of feature_desc has enhanced the understanding  
of the overall structure of the datasets and the meanings of the features,  
although some translations still remain unclear.  

Since there are a few features that have similar contents, it is necessary  
to perform imputation steps to resolve collinearity and reduce dimensionality  
before proceeding with the modeling process.

The following cells provide an overview of the number of features in each file  
and illustrates the feature intersection and difference between the files.

In [ ]:
feature_dict = {
    'azdias' : set(azdias.columns.unique()),
    'customers' : set(customers.columns.unique()),
    'ref_info' : set(ref_info.Attribute.dropna().unique()),
    'ref_attr' : set(ref_attr.Attribute.dropna().unique()),
    }

print ('Number of features by file')
for k, v in feature_dict.items():
    print (k, ':', len(v))

In [ ]:
# feature_dict['attr'] = set(attr.Attribute.dropna().unique())

print ('Feature intersection & difference between files')
for i in range (0, 4):
    for j in range (0, 4):
        if i == j:
            continue
        
        key_1 = list(feature_dict.keys())[i]
        key_2 = list(feature_dict.keys())[j]
        set_1 = list(feature_dict.values())[i]
        set_2 = list(feature_dict.values())[j]
        
        intsec = set_1.intersection(set_2)
        ft_diff = set_1 - set_2
        feature_dict[f'{key_1}_intsec_{key_2}'] = intsec
        feature_dict[f'in_{key_1}_notin_{key_2}'] = ft_diff
        
        intsec_print = f'{key_1} & {key_2} intersection:'
        ft_diff_print = f' /  in {key_1} & not-in {key_2}:'        
        print (intsec_print, str(len(intsec)).rjust(41 - len(intsec_print)),
               ft_diff_print, str(len(ft_diff)).rjust(40 - len(ft_diff_print)))

* Attribute of ref_info:너무 많은 피쳐가 reference 파일에 없음

In [ ]:
info_customers_similarity = similar_feature(
    'in_ref_info_notin_customers', 'in_customers_notin_ref_info', feature_dict)
info_customers_similarity.head()

In [ ]:
'''
많은 경우 _RZ 서픽스에 의한 차이이므로 info를 수정한
'''
org_info = ref_info.copy()
info_customers_similarity = info_customers_similarity.query('Attribute.str.len() > 5')

ref_col = 'in_customers_notin_ref_info'
ref_info = vlookup(ref_info, info_customers_similarity, lookup_col = ref_col)
ref_info.Attribute = np.where(ref_info[ref_col].isna(), ref_info.Attribute, ref_info[ref_col])
ref_info = ref_info.drop(ref_col, axis = 1)

* Attribute of ref_attr

In [ ]:
attr_customers_similarity = similar_feature(
    'in_ref_attr_notin_customers', 'in_customers_notin_ref_attr', feature_dict)
attr_customers_similarity.head()

In [ ]:
'''
많은 경우 _RZ 서픽스에 의한 차이이므로 attr를 수정한
'''
org_attr = ref_attr.copy()
attr_customers_similarity = attr_customers_similarity.query('Attribute.str.len() > 8')

ref_col = 'in_customers_notin_ref_attr'
ref_attr = vlookup(ref_attr, attr_customers_similarity, lookup_col = ref_col)
ref_attr.Attribute = np.where(ref_attr[ref_col].isna(), ref_attr.Attribute, ref_attr[ref_col])
ref_attr = ref_attr.drop(ref_col, axis = 1)

In [ ]:
feature_dict['ref_info'] = set(ref_info.Attribute.dropna().unique())
feature_dict['ref_attr'] = set(ref_attr.Attribute.dropna().unique())

print ('Feature intersection & difference between files')
for i in range (0, 4):
    for j in range (0, 4):
        if i == j:
            continue
        
        key_1 = list(feature_dict.keys())[i]
        key_2 = list(feature_dict.keys())[j]
        set_1 = list(feature_dict.values())[i]
        set_2 = list(feature_dict.values())[j]
        
        intsec = set_1.intersection(set_2)
        ft_diff = set_1 - set_2
        feature_dict[f'{key_1}_intsec_{key_2}'] = intsec
        feature_dict[f'in_{key_1}_notin_{key_2}'] = ft_diff
        
        intsec_print = f'{key_1} & {key_2} intersection:'
        ft_diff_print = f' /  in {key_1} & not-in {key_2}:'        
        print (intsec_print, str(len(intsec)).rjust(41 - len(intsec_print)),
               ft_diff_print, str(len(ft_diff)).rjust(40 - len(ft_diff_print)))

#### 2.2 reference file modification

<내용 수정 필요. 위 마크 업과 비교하여>
As datasets do not have information on what each column feature exactly means,  
values of reference files have to be mapped to the features.
  
feature_desc below is for this needs, and formed with 369 column features from customers. 

After merging information columns, there are 264 features from both customers and info,  
along with 105 exclusive features of customers lacking Description values.

* create feature_desc

In [ ]:
'''
feature_desc with column features of customers and added information from ref_info 
'''

feature_desc = pd.DataFrame(customers.columns, columns=['Attribute'])
feature_desc = vlookup(feature_desc, ref_info, 'Attribute')

'''
To fill missing Descriptions, add some values from attr and create translation using googletrans.
The result is saved in the root folder due to the time-consuming code execution.
'''
# # for Attribute without Description, add 12 Description values from ref_attr
# feature_desc = vlookup(feature_desc, ref_attr, 'Attribute', 'Description', intsec = 'update')

# # add category of Attribute
# feature_desc = attribute_cat(feature_desc)

# # add translated information to Desc
# feature_desc['eng_Desc'] = np.where(
#     feature_desc.Description.isna(),
#     feature_desc.Attribute.str.replace('_', ' ').apply(ger_to_eng),
#     np.nan)
# feature_desc['Desc'] = feature_desc.Description.fillna('') + feature_desc.eng_Desc.fillna('')

# # column order
# feature_desc = feature_desc[[
#     'category_large', 'category_small', 'Attribute', 'Information level', 'Additional notes', 'Description', 'Desc']]

feature_desc = pd.read_excel('feature_desc.xlsx', index_col = [0])
print(feature_desc.info())
feature_desc.head()

* clean ref_attr

In [ ]:
'''
change numbers in str form
'''
# 2 numbers in 1 str value
ref_attr.Score = ref_attr.Score.apply(
    lambda x: [int(i) for i in x.split(',')] if isinstance(x, str) and ',' in x else x)
ref_attr = ref_attr.explode('Score').reset_index(drop=True)
# '-1' to -1
ref_attr.Score = ref_attr.Score.apply(str_to_num)

'''
merge add-on information in some cells below Description
'''
# create attribute and description_shift with shift(-1)
ref_attr['attribute_shift'] = ref_attr.Attribute.shift(-1).fillna('')
ref_attr['description_shift'] = ref_attr.Description.shift(-1).fillna('')

# merge Description and add-on
ref_attr.Description = ref_attr.Description.mask(
    (~(ref_attr['Attribute'].isna()) & (ref_attr['attribute_shift'].isna())),
    ref_attr.Description + ' ' + ref_attr.description_shift)

# replace add-on with np.nan
ref_attr.Description = ref_attr.Description.mask(
    (ref_attr['Attribute'].isna()) & ~(ref_attr['Description'].isna()),
    np.nan)
# drop description shift
ref_attr = ref_attr.drop(['attribute_shift', 'description_shift'], axis = 1)

'''
fill blank cells with corresponding values as only 1st lines
of Attribute & Description have values
'''

ref_attr[['Attribute', 'Description']] = ref_attr[
    ['Attribute', 'Description']].fillna(method = 'ffill')

'''
keep 272 intersection features between ref_attr and datasets,
and drop exclusive features of datasets that are not useful at this step
'''

ref_attr = ref_attr[ref_attr.Attribute.isin(feature_dict['ref_attr_intsec_customers'])] # query not working

ref_attr = attribute_cat(ref_attr)
ref_attr = ref_attr[['category_large', 'category_small', 'Attribute', 'Description', 'Score', 'Meaning']]

print (f'number of Attribute: {ref_attr.Attribute.nunique()}')
ref_attr.head()

* create pre-study dataframes by value form of Score

In [ ]:
'''
add delimiter column Score_form
'''

ref_attr = ref_attr.copy() # to avoid SettingWithCopyWarning
ref_attr['Score_form'] = ref_attr.Score.map(type)
ref_attr.Score_form.value_counts()

In [ ]:
'''
attr_num containing Scores denoted in number
'''

attr_num = ref_attr[ref_attr.Score_form == int].copy()
print (f'number of Attribute(s): {attr_num.Attribute.nunique()}')

In [ ]:
'''
attr_str containing Scores denoted in string
'''

attr_str = ref_attr[ref_attr.Score_form == str].copy()
print (f'number of Attribute(s): {attr_str.Attribute.nunique()}')

In [ ]:
'''
there is 1 feature that Scores are in both numeric and string forms
'''

set(attr_num.Attribute.unique()).intersection(set(attr_str.Attribute.unique()))

#### 2.3 pre-study using reference files

After identifying the dataset structure in parts 2.1 and 2.2 above,  
part 2.3 focuses on examining the dataset contents.  
In this part, various aspects such as the ranges of feature values,  
values actually meaning null, data types, and more are to be verified,  
and preliminary notes will be recorded on pre-processing requirements.

<!-- ##### 2.3.0 attr modification -->

Considering that info contains information about dataset features, attr contains  
information about dataset values and the values within the datasets can be understood  
by mapping them to the corresponding Meaning values in attr.

However, it is necessary to initially modify attr since it is not in a neat form:
* 2 numbers in 1 str value like '-1, 0' or number(s) in str like '-1'
* add-on information cell below Description value cell
* only 1st rows of Attribute & Description with values, etc.

##### 2.3.1 ref_attr Scores denoted in number

먼저 attr의 contents를 보면 대부분의 데이터는 categorical 하며, 좀 더 효과적으로 데이터의 Score를 살펴보기 위해 각 Attribute의 min/max 값을 카테고리화 해 본 결과 아래와 같이 나왔다. 

< 아래 내용은 나중에 생사 >  
From min/max categories 를 만들었으며, to be further checked are:   
* If Score -1 and 0 contain data equal to null or -inf  
* If max Score values contain data equal to null or inf
* If features with wider min/max gap are discrete or continuous

In [ ]:
'''
summary of attr_num with min/max value observed
'''

attr_num_score = pv_min_max(attr_num, feature_desc)
attr_num_score.head()

* **data type of features with wider min/max Score gap**
  
attr의 scoring 체계가 주어졌거니와 max/min 차이를 보면 대부분의 데이터가 discrete 임을 알 수 있음.  
그러나 min/max gap이 10 이상이면 continuous 일 수 있으므로 살펴 보았는데 이 또한 All features that have min/max Score gap over 10 are found to be discrete

In [ ]:
'''
check features that have min/max Score gap over 10
'''
# from attr_num_score above, extract Attribute values having min/max Score gap over 10
temp_over10_attribute = attr_num_score[
    (attr_num_score['max'] - attr_num_score['min']) > 10].Attribute.unique()

temp_over10 = pv_meaning_score(
    attr_num.query('Attribute in @temp_over10_attribute'), feature_desc)

* Meaning 'unknown'

In [ ]:
'''
어떤 값이 unknown을 갖나?
'''

ref_attr.query('Meaning == "unknown"').Score.unique()

In [ ]:
'''
when Score value is -1, the meaning is 100% unknown in reference file
따라서 dataset에서도 -1은 모두 null 처리가 reasonable함
'''

attr_num.query('Score < 0').Meaning.value_counts()

In [ ]:
'''
NOTE dict p_process to note points to be pre-processed
'''

p_process = {'2.3.1-minus1' : 'replace / All / {-1: np.nan}'}

In [ ]:
'''
-1이 아닌 경우는 null인 경우와 아닌 경우가 혼재
따라서 위에서 정리한 Attribute의 Category 기반으로 0 이나 9인
datasets Score 중 반드시 null로 처리할 것이 있는지 파악해 본다.
'''
print (attr_num.query('Score == 0').Meaning.unique())
print (attr_num.query('Score == 9').Meaning.unique())

In [ ]:
'''
먼저 0의 경우 Meaning == 'unknown'인 category_large는
'''

unknown_0 = attr_num.query('Score == 0 and Meaning == "unknown"').category_large.unique()
unknown_0

In [ ]:
'''
there category_large에서는 Score 0이더라도 Meaning != "unknown"이 발견되므로
Score 0 이라 해도 일괄적으로 null을 적용할 수는 없음
'''

attr_num.query(
    'category_large in (@unknown_0) and Score == 0 and Meaning != "unknown"').head()

In [ ]:
'''
0의 경우 Meaning == 'unknown'인 category_small은
'''

unknown_9 = attr_num.query('Score == 9 and Meaning == "unknown"').category_small.unique()
unknown_9

In [ ]:
'''
there category_small에서는 Score 9이면 100% Meaning == "unknown"이므로
동 category의 Score 9는 모두 null 처리함
'''

attr_num.query('category_small in (@unknown_9) and Score == 9 and Meaning != "unknown"')

In [ ]:
'''
NOTE pre-processing items
'''

p_process['2.3.1-unknown_9'] = 'mask / category_small in unknown_9 / Score 9 to -1'

* 언노운 외의 null에 상응하는 value는 없는가?

In [ ]:
'''
최소값 -1은 모두 null이며, 최소값 0 또는 1일 경우

아래 내용은 나중에 생사
when Score value is 0, it means none in most cases,
and there are also some corresponding Meaning values
that seem to be definitely equal to null
'''

# 최소값 0 Meaning 점검
temp_min_0 = attr_num_score[attr_num_score['min'] == 0].Attribute
print (attr_num.query('Attribute in (@temp_min_0) and Score == 0').Meaning.unique())

# 최소값 -1 Meaning 점검
temp_min_1 = attr_num_score[attr_num_score['min'] == 1].Attribute
print (attr_num.query('Attribute in (@temp_min_1) and Score == 1').Meaning.unique())

In [ ]:
'''
먼저 확실하게 null을 의미하는 2개 Score는 to_null 리스트에 넣어 later, pre-processing에 사용하고, 
context에 따라 정확한 의미 파악이 필요한 나머지 값들은 maybe_null에 넣어 상세 점검을 함
'''
to_null = ['unknown', 'unknown / no main age detectable']
maybe_null = ['no transaction known', 'no transactions known', 'none']

'''
메이비널 값을 컨텍스트 속에서 살펴보면
- maybe_null은 은 null이 아니며 0을 의미함
'''
for i in maybe_null:
    print (i)
    verify_maybe_null = pv_verify_null(attr_num, feature_desc, i)

In [ ]:
'''
NOTE pre-processing items
'''

p_process['2.3.1-1'] = 'mask / All / replace Score to null if corresponding Meaning is in to_null'

##### 2.3.2 ref_attr Scores denoted in string

///// 98 features in str type are analyzed

In [ ]:
'''
summary of attr_str with Meaning and Score values
'''

attr_str_score = pv_meaning_score(attr_str, feature_desc, display = False)
view_all(attr_str_score)

* **7 features are found as continuous.**  
  These features will checked in detail in the next section.

In [ ]:
# create list of possibly containing continuous features 
pre_study_continuous = list(
    attr_str_score[attr_str_score.Score.apply(lambda x: x == ['…'])].Attribute.unique())

### 3. Dataset exploration

Based on baseline understanding and frame works from reference files in section_2,  
section_3 is dealing with actual datasets.  
In this section, steps taken in the previous section will be repeated to fine-tune  
and finalize pro-processing items.

In [ ]:
'''
create a concatenated dataframe of 2 datasets for temporary use
'''

concat_data = pd.concat([customers, azdias], axis=0).reset_index(drop = True)
concat_data.info()

In [ ]:
'''
datasets containing intersection features in both of datasets and ref_attr
'''

datasets = concat_data[
    list(feature_dict['customers_intsec_ref_attr'])].copy()
datasets.shape

In [ ]:
'''
datasets_excl containing exclusive features of datasets
'''

datasets_excl = concat_data[
    list(feature_dict['in_customers_notin_ref_attr'])].copy()
datasets_excl.shape

#### 3.1 intersection features in both of datasets and ref_attr

##### 3.1.1 numeric dataset Scores of intersection features

In [ ]:
'''
datasets_num_score containing summary of numeric Score values
'''

datasets_num_score = dataset_summary(datasets, feature_desc, ['int', 'float'])

In [ ]:
'''
check if Scores in actual datasets are within range given by ref_attr.
'''

# merge min/max values by Attribute in attr_num_score in 2.3.1
temp_range_check = vlookup(
    datasets_num_score, attr_num_score, 'Attribute', ['min', 'max', 'min_max_cat'], intsec = 'copy')

temp_range_check['check'] = np.where(
    temp_range_check['min_max_cat_'].isnull(), 
    'not in attr_num',
    np.where(
        (temp_range_check['min'] < temp_range_check['min_']),
         'min lower',
         np.where((temp_range_check['max'] > temp_range_check['max_']),
         'max higher',
         'within min/max range')))

temp_range_check.check.value_counts()

* features out of Score range of ref_attr  

293 피쳐는 pre_study에서 살펴본 Range 안에 Score가 위치하므로 Section2에서 정리한 pre-processing을 진행하면 됨

레인지를 벗어난 피쳐에 대해서는 아래에서 살펴보고 처리방안을 정리함. 또한 피쳐 이름에서 데이터 유사성이 발견되어 유사 데이터는 드롭하기로 함.

In [ ]:
'''
4 features are showing lower min value in dataset 
'''
temp_range_check.query('check == "min lower"')

In [ ]:
'''
these 4 features of datasets have 0 as min Score.
as Meaning values of these 4 features in ref_attr do not cover null value,
Score 0 should be replaced as null (see example below)
'''

ref_attr.query('Attribute == "LP_LEBENSPHASE_GROB"').Meaning.unique()

In [ ]:
'''
NOTE pre-processing items
'''

for i in temp_range_check.query('check == "min lower"').Attribute.unique():
    p_process['3.1.1-'+ i] = 'replace / ' + i + ' / {0: -1}'

In [ ]:
'''
위 피쳐의 이름을 보면 gross와 fine으로 나뉘어 있으며, 같은 피쳐에 그로스/파인이 들어 있는 항목은
유사피쳐를 크게 나누느냐 작게 나누느냐의 차이임
'''
ref_attr.query('Attribute.str.contains("LP_FAMILIE")')
# attribute_without_meaning[attribute_without_meaning.Score > 8]

In [ ]:
'''
유사한 gross와 fine이 있는 피쳐는
'''
suffix_gross_fine = ['GROB', 'FEIN']
features_gross_fine = feature_desc[
    feature_desc.Attribute.str.split('_').str[-1].isin(suffix_gross_fine)]
features_gross_fine = list(features_gross_fine.Attribute.unique())
features_gross_fine

In [ ]:
'''
유사한 gross가 있는 피쳐만 list로 정리하여 drop 함
'''

to_drop = feature_desc.query('Attribute.str.contains("GROB")')
to_drop = list(to_drop.Attribute.unique())

'''
NOTE pre-processing items
'''

p_process['2.3.1-drop'] = 'drop / All / drop features in to_drop'

* continuous features:   
  뭐뭐뭐

In [ ]:
'''
7 features are continuous:
in ref_attr, Score values of these continuous features have string form (see 2.3.2) 
'''
continuous_in_datasets = temp_range_check.query('check == "not in attr_num"')
print (set(continuous_in_datasets.Attribute.unique()) == set(pre_study_continuous))
continuous_in_datasets

In [ ]:
'''
ANZ_PERSONEN  
- Value 0 might be strange or extreme but is possible
- outliers: household with over 10 adult persons is highly extreme or data error,
  that Value over 10 should be replaced to NaN
'''

view_feature (datasets, feature_desc, 'ANZ_PERSONEN', False)

In [ ]:
'''
NOTE
'''

p_process['3.1.1-ANZ_PERSONEN'] = 'np.where / ANZ_PERSONEN / Score > 10 to -1'

In [ ]:
'''
ANZ_TITEL
- it seems that there is no pre-processing needed.
  However, this feature is linked to ANZ_PERSONEN above
  that Score should be replaced to null if corresponding ANZ_PERSONEN is null.
'''

view_feature (datasets, feature_desc, 'ANZ_TITEL', False)

In [ ]:
p_process['3.1.1-ANZ_TITEL'] = 'np.where / ANZ_TITEL / Score of ANZ_PERSONEN > 10 to -1'

In [ ]:
'''
ANZ_HH_TITEL  
- No pre-processing needed: max values might be strange or extreme but are possible
'''

view_feature (datasets, feature_desc, 'ANZ_HH_TITEL', False)

In [ ]:
'''
KBA13_ANZAHL_PKW
- in this feature values over 1250 is grouped by 100. 
  Log scaling is needed due to high skewness.
'''

view_feature (datasets, feature_desc, 'KBA13_ANZAHL_PKW')

In [ ]:
p_process['3.1.1-KBA13_ANZAHL_PKW'] = 'log_scaling / KBA13_ANZAHL_PKW'

In [ ]:
'''
MIN_GEBAEUDEJAHR   
- No pre-processing needed
'''

view_feature (datasets, feature_desc, 'MIN_GEBAEUDEJAHR', True, False)

In [ ]:
'''
GEBURTSJAHR  
- outliers: birth year can not be 0 and anyone born before 1900 is extreme
  that Value < 1900 should be replaced to null
'''

view_feature (datasets, feature_desc, 'GEBURTSJAHR')

In [ ]:
p_process['3.1.1-GEBURTSJAHR'] = 'np.where / GEBURTSJAHR / Score < 1900 to -1'

In [ ]:
'''
ANZ_HAUSHALTE_AKTIV  
- No pre-processing needed: Value 0 and max value might be strange or extreme but are possible
- Log scaling is needed due to high skewness
'''

view_feature (datasets, feature_desc, 'ANZ_HAUSHALTE_AKTIV')

In [ ]:
'''
NOTE
'''

p_process['3.1.1-ANZ_HAUSHALTE_AKTIV'] = 'log_scaling / HAUSHALTE_AKTIV'

##### 3.1.2 object dataset Scores of intersection features

* 뭐뭐뭐

In [ ]:
datasets_obj_score = dataset_summary(datasets, feature_desc, ['object'])

In [ ]:
'''
check if Scores in actual datasets are within range given by attr.
'''

# merge min/max values by Attribute in attr_num_score in 2.3.1
for i in datasets_obj_score.Attribute.unique():
    score_in_datasets = set(datasets[i].unique())
    score_in_ref_attr = set(ref_attr.query('Attribute == "{}"'.format(i)).Score.values)
    
    print (i)
    print (f'a) scores in datasets: {score_in_datasets}')
    print (f'b) scores in ref_attr: {score_in_ref_attr}')
    print (f'a) b) intersection: {score_in_datasets.intersection(score_in_ref_attr)}')
    print (f'a) - b): {score_in_datasets - score_in_ref_attr}')
    print (f'b) - a): {score_in_ref_attr - score_in_datasets}', '\n')

In [ ]:
'''
피쳐 2개는 Score가 추치가 아니며 나머지 2개는 수치 Score를 string으로 기록한 경우가 있으며
'XX', 'X'로 null이 표기되어 있음
'''

none_numeric_in_datasets = ['CAMEO_DEU_2015', 'OST_WEST_KZ']

'''
NOTE
'''

p_process['3.1.2-CAMEO_INTL_2015XX'] = 'replace / CAMEO_INTL_2015 / {"XX": -1}'
p_process['3.1.2-CAMEO_DEU_2015'] = 'replace / CAMEO_DEU_2015 / {"XX": -1}'
p_process['3.1.2-CAMEO_DEUG_2015X'] = 'replace / CAMEO_DEUG_2015 / {"X": -1}'
p_process['3.1.2-dtype'] = 'astype / All features in numeric_discrete_features / change dtype to float'

#### 3.2 exclusive features NOT in reference files

In [ ]:
'''
datasets_excl containing exclusive features of datasets
'''

datasets_excl = concat_data[
    list(feature_dict['in_customers_notin_ref_attr'])].copy()

##### 3.2.1 numeric dataset Scores of exclusive features

뭐뭐뭐

In [ ]:
'''
excl_num_score containing Scores denoted in number
'''

datasets_excl_num_score = dataset_summary(datasets_excl, feature_desc, ['int', 'float'])

* **data type of features with wider min/max Score gap**
  
min/max Score gap is not big. 아무 reference가 없으니 section_2에서 정리한 일반 원칙 적용하여 pre-processing

All features that have min/max Score gap over 10 and null portion below 50% are to be checked below.

In [ ]:
'''
check features that have min/max Score gap over 10
나머지 피쳐 중 드롭한 50% 이상 빈 피쳐를 제외하면 6개가 컨티뉴어스임 
'''
# from datasets_excl_num_score above, extract Attribute values having min/max Score gap over 10
continuous_in_datasets_excl_attribute = datasets_excl_num_score[
    (datasets_excl_num_score['max'] - datasets_excl_num_score['min']) > 10].Attribute.unique()

continuous_in_datasets_excl = datasets_excl_num_score.query(
    'Attribute in @continuous_in_datasets_excl_attribute and null_portion < 0.5')
view_all(continuous_in_datasets_excl)

In [ ]:
'''
디스크리트인 2개는 디스크립션을 보면 카테고리임
'''
continuous_in_datasets_excl = continuous_in_datasets_excl.query(
    'Attribute != "ALTERSKATEGORIE_FEIN" and Attribute != "GEMEINDETYP"')

In [ ]:
'''
NOTE
LNR is the serial index of dataset 이므로 드롭함
'''

to_drop.append('LNR')

In [ ]:
'''
ANZ_STATISTISCHE_HAUSHALTE   
  - No pre-processing needed: Value 0 and max value might be strange or extreme but are possible
  - Log scaling is needed due to high skewness
'''

view_feature (datasets_excl, feature_desc, 'ANZ_STATISTISCHE_HAUSHALTE')

In [ ]:
'''
NOTE
'''

p_process['3.2.1-ANZ_STATISTISCHE_HAUSHALTE'] = 'log_scaling / ANZ_STATISTISCHE_HAUSHALTE'

In [ ]:
'''
ANZ_KINDER  
  - 0이 매우 많기는 하나 정확한 뜻을 알 수 없으니 살림
'''

view_feature (datasets_excl, feature_desc, 'ANZ_KINDER', False, True)

In [ ]:
'''
VK_DISTANZ  
- No pre-processing needed although Eng translation not completely understandable
'''

view_feature (datasets_excl, feature_desc, 'VK_DISTANZ', False, True)

In [ ]:
'''
VERDICHTUNGSRAUM   
- No pre-processing needed although Eng translation not completely understandable
'''

view_feature (datasets_excl, feature_desc, 'VERDICHTUNGSRAUM')

In [ ]:
'''
EINGEZOGENAM_HH_JAHR 
- outliers: Eng translation is not completely understandable, but 3 outliers can be replaced to null
'''

view_feature (datasets_excl, feature_desc, 'EINGEZOGENAM_HH_JAHR', True, False)

In [ ]:
p_process['3.2.1-EINGEZOGENAM_HH_JAHR'] = 'np.where / EINGEZOGENAM_HH_JAHR / Score < 1980 to -1'

##### 3.2.2 object dataset Scores of exclusive features

///// For ....

In [ ]:
'''
datasets_excl_obj_score containing Scores denoted in obj
'''

datasets_excl_obj_score = dataset_summary(datasets_excl, feature_desc, ['object'])

In [ ]:
'''
4개 모두 디스크리트임
'''

none_numeric_in_datasets_excl = ['D19_LETZTER_KAUF_BRANCHE', 'CUSTOMER_GROUP', 'EINGEFUEGT_AM', 'PRODUCT_GROUP']

In [ ]:
'''
NOTE
datetime으로 변환하데 년도만 남김 as there are too many unique values
'''

p_process['3.2.2-EINGEFUEGT_AM'] = 'astype / EINGEFUEGT_AM / change dtype to datetime only with year values'

In [ ]:
'''
check if abnormal Scores exist in actual datasets.
으나 전처리 내용은 없음
'''
# 제외
datasets_excl_obj_score = datasets_excl_obj_score.query('Attribute != "EINGEFUEGT_AM"')

# 
for i in datasets_excl_obj_score.Attribute.unique():
    
    print (i)
    print (f'scores in datasets: {set(datasets_excl[i].unique())}', '\n')

### 4. Pre-processing

* categorize pre-processing items

In [ ]:
'''
create pre-processing items
'''
# transform p_process from dictionary to dataframe 
p_process_items = {'pre_processing_itmes' : p_process}
p_process_items = pd.DataFrame(p_process_items).reset_index()
p_process_items [['method', 'by_feature']] = p_process_items[
    'pre_processing_itmes'].str.split(' / ', n=1, expand=True)
p_process_items = p_process_items[['index', 'method', 'by_feature']].sort_values(['method', 'index'])
view_all(p_process_items)

* create pre-processing reference

In [ ]:
'''
replacements dictionary
'''
p_process_replace = p_process_items.query('method == "replace"').iloc[1:]
p_process_replace[['col', 'dict_val']] = p_process_replace[
    'by_feature'].str.split(' / ', expand=True)

# replacements = dict(zip(p_process_replace['col'], p_process_replace['dict_val']))
replacements = dict(map(lambda col, dict_val: (col, eval(dict_val)), 
                        p_process_replace['col'], p_process_replace['dict_val']))
replacements

In [ ]:
'''
continuous and discrete features
'''

# continuous_features form 3.1.1 and 3.2.1
continuous_features = list(
    continuous_in_datasets.Attribute.unique()) + list(continuous_in_datasets_excl.Attribute.unique())

# discrete_features
discrete_features = list(x for x in datasets.columns if x not in continuous_features)
none_numeric_discrete_features = none_numeric_in_datasets + none_numeric_in_datasets_excl
numeric_discrete_features = list(x for x in discrete_features if x not in none_numeric_discrete_features + to_drop)

# save lists as pickle
for i, j in zip(
    [continuous_features, discrete_features, none_numeric_discrete_features, numeric_discrete_features],
    ['continuous_features', 'discrete_features', 'none_numeric_discrete_features', 'numeric_discrete_features']):
    
    pickle_list = pickle.dumps(i)
    with open(j + '.pkl', 'wb') as f:
        f.write(pickle_list)

In [ ]:
for i, j in zip(
    [continuous_features, discrete_features, none_numeric_discrete_features, numeric_discrete_features],
    ['continuous_features', 'discrete_features', 'none_numeric_discrete_features', 'numeric_discrete_features']):
    
    pickle_list = pickle.dumps(i)
    with open(j + '.pkl', 'wb') as f:
        f.write(pickle_list)

In [ ]:
'''
mask to process Scores corresponding to Meaning in to_null
'''

mask_to_null = ref_attr.query('Meaning in (@to_null)')[['Attribute', 'Score']].set_index('Attribute')
mask_to_null = mask_to_null[mask_to_null.Score != -1]

features_unknown_9 = [i for i in feature_desc.Attribute.unique() if i.split('_')[0] in unknown_9]
mask_unknown_9 = pd.DataFrame(index = features_unknown_9, columns = ['Score'], data = 9)

* pre-processing

In [ ]:
view_all(p_process_items)

In [ ]:
def pre_processing (data):
    '''
    function to pre_process datasets
    '''        
    # drop
    data = data.drop(to_drop, axis = 1)

    # mask
    mask = data.eq(mask_to_null['Score'], axis=1)
    data = data.mask(mask)
    
    mask = data.eq(mask_unknown_9['Score'], axis=1)
    data = data.mask(mask)

    # np.where
    data['ANZ_PERSONEN'] = np.where(data['ANZ_PERSONEN'] > 10, -1, data['ANZ_PERSONEN'])
    data['ANZ_TITEL'] = np.where(data['ANZ_PERSONEN'] > 10, -1, data['ANZ_TITEL'])
    data['GEBURTSJAHR'] = np.where(data['GEBURTSJAHR'] < 1900, -1, data['GEBURTSJAHR'])
    data['EINGEZOGENAM_HH_JAHR'] = np.where(data['EINGEZOGENAM_HH_JAHR'] < 1980, -1, data['EINGEZOGENAM_HH_JAHR'])

    # replacement
    data = data.replace(replacements)
    data = data.replace(-1, np.nan)

    # astype
    data[numeric_discrete_features] = data[numeric_discrete_features].astype(float)  
    data['EINGEFUEGT_AM'] = pd.to_datetime(data['EINGEFUEGT_AM'], format = '%Y-%m-%d').dt.year 

    return data

In [ ]:
org_azdias = azdias.copy()
azdias = pre_processing (azdias)

In [ ]:
org_customers = customers.copy()
customers = pre_processing (customers)

* drop columns and rows with missing values over threshold

In [ ]:
'''missing column overview: azdias'''

azdias.name = 'azdias'
miss_val_summary (azdias)

In [ ]:
'''missing column overview: customers'''

customers.name = 'customers'
miss_val_summary (customers)

In [ ]:
'''
columns of azdias with missing values over 33%
'''
temp_missing_col = azdias.isnull().mean()
azdias_missing_col = list(temp_missing_col[temp_missing_col > 0.33].index)
print ('azdias missing columns:', azdias_missing_col)

temp_missing_col = customers.isnull().mean()
customers_missing_col = list(temp_missing_col[temp_missing_col > 0.33].index)
print ('customers missing columns:', customers_missing_col)

In [ ]:
'''drop columns with missing values over 33%'''

azdias = azdias.drop(azdias_missing_col, axis = 1)
customers = customers.drop(customers_missing_col, axis = 1)

In [ ]:
'''missing row overview: azdias'''

azdias.name = 'azdias'
miss_val_summary (azdias, axis_val = 'row', bar_chart = False)

In [ ]:
'''missing row overview: customers'''

customers.name = 'customers'
miss_val_summary (customers, axis_val = 'row', bar_chart = False)

In [ ]:
'''drop columns with missing values over 33%'''

temp_missing_row = azdias.isnull().sum(axis = 1) / azdias.shape[1]
temp_missing_row = temp_missing_row > 0.33
azdias = azdias[~temp_missing_row]

temp_missing_row = customers.isnull().sum(axis = 1) / customers.shape[1]
temp_missing_row = temp_missing_row > 0.33
customers = customers[~temp_missing_row]

* save pre-processed files as parquet

In [ ]:
table = pa.Table.from_pandas(azdias)
pq.write_table(table, 'azdias.parquet')

In [ ]:
table = pa.Table.from_pandas(customers)
pq.write_table(table, 'customers.parquet')

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

In [2]:
'''load pre-preocessed datasets'''

azdias = pq.read_table('azdias.parquet').to_pandas()
azdias.shape

(785421, 350)

In [3]:
customers = pq.read_table('customers.parquet').to_pandas()
customers.shape

(140400, 353)

In [5]:
'''load lists of feature category'''
for i in ['continuous_features', 'discrete_features', 'none_numeric_discrete_features', 'numeric_discrete_features']:
    
    with open(i + '.pkl', 'rb') as f:
        pickle_list = f.read()

    globals()[i] = pickle.loads(pickle_list)

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

Work / Ref

In [ ]:
# def col_val (df):
#     '''
#     function to check values of dataframe columns
#     df : dataframe
#     '''
#     # for i in df.columns:
#     #     print (i, '-', df[i].nunique(), 'values', '\n',
#     #         df[i].value_counts(), '\n', '*     *     *')
#     for i in df.columns:
#         print (i, '-', df[i].nunique(), 'values', '\n',
#         list(df[i].unique()), '\n', '*     *     *')

In [ ]:
# # from googletrans import Translator

# # Create an instance of the Translator
# translator = Translator(service_urls=['translate.google.com'])

# # Text to be translated
# text = "AGER_TYP"

# # Translate the text from German to English
# translation = translator.translate(text, src='de', dest='en')

# # Print the translated text
# print("Original text (German):", text)
# print("Translated text (English):", translation.text)

In [ ]:
# def miss_val_hist(df, axis_val, x_bin = 10):
#     '''
#     function to display missing value histogram by column or raw
#     df: dataframe
#     axis_val: str, one of 'column' or 'row'
#     x_bin: size of xtick bin, 10 as default 
#     '''
#     # axis value
#     axis_num = 0 if axis_val == 'column' else 1
    
#     # % of missing values
#     missing_pct = df.isnull().mean(axis = axis_num) * 100

#     # max % of missing values by column
#     missing_pct_max = missing_pct.max()
#     print ('max % of missing values by ' + axis_val + ': ', missing_pct_max)

#     # plot missing values by column
    
#     print (missing_pct.describe())
    
#     x_range = ((missing_pct_max + x_bin * 2) // x_bin) * x_bin

#     ax = missing_pct.plot(
#         kind = 'hist', figsize=(10, 3), color='gray',
#         bins = np.arange(0, x_range, 10),
#         title = (df.name + ': missing value by ' + axis_val)
#         )
#     ax.set_xlabel('% of missing value');

In [ ]:
# 파이써닉 하지 못한 코드
# # missing value overview
# for i in range(0, ((azdias.shape[1] + 100) // 100) * 100, 100):
#     msno.matrix(azdias.iloc[:, i : i + 99],
#                 figsize=(10, 3), fontsize = 12, labels = False, sparkline = False)
#     plt.title('missing value overview: col ' + str (i) + ' to ' + str (min(i + 99, azdias.shape[1] - 1)),
#               fontsize = 12);

In [ ]:
# # % of columns with missing values of over 30%
# (azdias.isnull().mean() * 100 > 30).mean()

In [ ]:
# # % of columns with missing values of 25% to 30%
# ((azdias.isnull().mean() * 100 > 25) & (30 >= azdias.isnull().mean() * 100)).mean()

In [ ]:
# 아마도 쓰지 않을 plot
# plot = azdias_col_missing_pct.plot(
#     kind = 'bar', figsize=(10, 3), color='dimgray', xticks = [],
#     title = 'azdias_col_missing_pct',
#     xlabel = '366 columns',
#     ylabel = '% of missing values')

In [ ]:
# num_df = customers.select_dtypes(include=['float', 'int64']).iloc[:, 1:]
# num_df.head()

In [ ]:
# len(pd.unique(customers.select_dtypes(include='float').values.flatten()).tolist())

In [ ]:
# with pd.option_context(
#     'display.max_rows', None, 'display.max_colwidth', None):
#     display(pd.DataFrame(attr.apply(lambda x: x.unique())))

In [ ]:
# attr_not_null = ~(attr.Attribute.isna())
# attr.loc[attr_not_null, 'Description'] = attr.loc[
#     attr_not_null, 'Description'] + ' ' + attr.loc[attr_not_null, 'desc_shift']

# desc_to_null = (attr.Attribute.isna()) & ~(attr.Description.isna())
# attr.loc[desc_to_null, 'Description'] = np.nan
# attr = attr.drop(columns = 'desc_shift')
# attr.loc[attr_with_value.shift(-1, fill_value = True), 'Description']
# attr_shift = attr_null.shift
# attr[attr_null.shift, 'Description'] = attr.loc[
#     attr_null.shift(fill_value = False), 'Description'] + ' ' + attr[attr_null, 'Description']

In [ ]:
# '''
# to 313 rows of Attribute in info, add 15 values exclusively in attr,
# and remove 52 values exclusively in information files
# '''

# info_mg = info.iloc[:, 1:].copy()
# info_mg = info_mg.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
# # attr_excl = attr[attr.Attribute.isin(attr_excl)].copy()
# attr_not_null = attr.dropna(subset = 'Attribute').copy()
# attr_not_null = attr_not_null.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

# info_mg = pd.concat(
#     [info_mg, attr_not_null[['Attribute', 'Description']]],
#     ignore_index  = True,
#     axis = 0
#     )
# info_mg = info_mg.drop_duplicates(subset = ['Attribute', 'Description'])
# info_mg = info_mg.sort_values(by = list(info_mg.columns), ascending=False)
# # info_mg = info_mg.drop_duplicates(subset='Attribute')

# info_mg = info_mg[~(info_mg.Attribute.isin(infofile_excl))]

# print(info_mg.info())
# info_mg.head()

In [ ]:
# '''
# to 313 rows of Attribute in info, add 15 values exclusively in attr,
# and remove 52 values exclusively in information files
# '''

# info_mg = info.iloc[:, 1:].copy()
# attr_excl = attr[attr.Attribute.isin(attr_excl)][['Attribute', 'Description']].copy()

# info_mg = pd.concat(
#     [info_mg, attr_excl],
#     ignore_index  = True,
#     axis = 0
#     )
# info_mg = info_mg.drop_duplicates(subset = ['Attribute', 'Description'])
# # info_mg = info_mg.sort_values(by = list(info_mg.columns), ascending=False)
# # # info_mg = info_mg.drop_duplicates(subset='Attribute')

# info_mg = info_mg[~(info_mg.Attribute.isin(infofile_excl))]

# print(info_mg.info())
# info_mg.head()

In [ ]:
# view_all(info_mg)

In [ ]:
# '''
# to 2258 rows of Attribute in attr, add 13 values exclusively in info,
# and remove 52 values exclusively in information files
# '''

# attr_mg = attr.copy()
# info_excl = info[info.Attribute.isin(info_excl)][['Attribute', 'Description']].copy()
# info_excl['Value'] = 'form info'
# info_excl['Meaning'] = 'form info'

# attr_mg = pd.concat(
#     [attr_mg, info_excl],
#     ignore_index  = True,
#     axis = 0
#     )
# # info_mg = info_mg.drop_duplicates()

# # info_mg = info_mg[~(info_mg.Attribute.isin(infofile_excl))]

# print(attr_mg.info())
# attr_mg.head()

In [ ]:
# attr_mg.tail(30)

In [ ]:
# '''
# there are 93 and 51 exclusive values in data and information files
# '''

# datafile_Attr = azdias_Attr.union(customers_Attr)
# infofile_Attr = info_Attr.union(attr_Attr)

# datafile_excl = datafile_Attr - infofile_Attr
# infofile_excl = infofile_Attr - datafile_Attr

# print (len(datafile_excl), 'Attribute value(s) exclusively in data files:',
#        '\n', datafile_excl)
# print (len(infofile_excl), 'Attribute value(s) exclusively in information files:',
#        '\n', infofile_excl)

In [ ]:
# 불필요 한 것으로 보임
# '''
# fill null cells as only 1st lines of information have values
# '''

# info['Information level'] = info['Information level'].fillna(method = 'ffill')

# info.head()

In [ ]:
# 불필요 한 것으로 보임
# '''
# fill null cells as only 1st lines of information have values
# '''

# attr[['Attribute', 'Description']] = attr[
#     ['Attribute', 'Description']].fillna(method = 'ffill')

# attr.head(10)

In [ ]:
# '''
# To enhance readability of data set - over 100 features are without description -   
# I added 14 Description values from attr, and made a column of translation   
# (ger_to_eng) to df_feature using code below.
# However as this code-running takes somewhat long time, I saved the dataframe processed
# as df_feature.xlsx in root folder
# '''

# # For values of Attribute without Description, add 14 Description values from attr
# df_feature.set_index('Attribute', inplace = True)

# attr_excl = attr[attr.Attribute.isin(attr_Attr - info_Attr)][['Attribute', 'Description']].copy()
# attr_excl.set_index('Attribute', inplace = True)
# df_feature.update(attr_excl)

# df_feature.reset_index(inplace = True)

# # For values of Attribute without Description, make colum of translation (ger_to_eng)
# def ger_to_eng (ger_text):
#     '''
#     function to translate German text
#     '''    
#     translator = Translator(service_urls=['translate.google.com'])    
#     try:
#         translation = translator.translate(ger_text, src='de', dest='en')
#         return translation.text        
#     except:
#         return np.nan

# df_feature['ger_to_eng'] = np.where(
#     df_feature.Description.isnull(),
#     df_feature.Attribute.str.replace('_', ' ').apply(ger_to_eng),
#     np.nan)
# df_feature['Desc'] = df_feature.Description.fillna('') + df_feature.ger_to_eng.fillna('')

# # # sort by Attribute and Information level
# # df_feature.sort_values(by = ['Attribute', 'Information level'], inplace= True)

# # df_feature = pd.read_excel('df_feature.xlsx', index_col = [0])
# # df_feature.head(10)

In [ ]:
# attr_excl = attr[attr.Attribute.isin(attr_Attr - info_Attr)][['Attribute', 'Description']].copy()
# df_feature.Description = df_feature.Description.mask(
#     df_feature.Attribute == attr_excl.Attribute,
#     attr_excl.Description
#     )
# print ('Attributes missing Description:', df_feature[df_feature.Description.isna()].shape[0])
# print (df_feature.shape)
# df_feature.head()

# ValueError: Can only compare identically-labeled Series objects

In [ ]:
# feature_desc.set_index('Attribute', inplace = True)

# attr_Attr = set(attr.Attribute.dropna().unique())
# info_Attr = set(info.Attribute.dropna().unique())
# attr_excl = attr[
#     attr.Attribute.isin(attr_Attr - info_Attr)][['Attribute', 'Description']].copy()
# attr_excl.set_index('Attribute', inplace = True)
# feature_desc.update(attr_excl)

# feature_desc.reset_index(inplace = True)

In [ ]:
# '''
# alignment of features between data files: 
# df customers has 3 more exclusive columns
# '''

# azdias_Attr = set(azdias.columns)
# customers_Attr = set(customers.columns)

# print(azdias_Attr - customers_Attr)
# print(customers_Attr - azdias_Attr)

In [ ]:
# '''
# alignment of features between information files
# '''
# info_Attr = set(info.Attribute.dropna().unique())
# attr_Attr = set(attr.Attribute.dropna().unique())

# # info_excl = info_Attr - attr_Attr
# # attr_excl = attr_Attr - info_Attr

# print (len(info_Attr - attr_Attr), 'Attribute value(s) exclusively in info:',
#        '\n', info_Attr - attr_Attr)
# print (len(attr_Attr - info_Attr), 'Attribute value(s) exclusively in attr:',
#        '\n', attr_Attr - info_Attr)
# '''
# alignment of features between customers and information files
# '''
# print ('Attribute between customers and info')
# print (len(customers_Attr - info_Attr), 'feature(s) exclusively in customers:',
#        '\n', customers_Attr - info_Attr)
# print (len(info_Attr - customers_Attr), 'Attribute value(s) exclusively in info:',
#        '\n', info_Attr - customers_Attr)
# print ('In', len(info_Attr), 'features of info,', 
#        len(info_Attr) - len(info_Attr - customers_Attr), 'features are in Attribute of customers', '\n')

# print ('Attribute between customers and attr')
# print (len(customers_Attr - attr_Attr), 'feature(s) exclusively in customers:',
#        '\n', customers_Attr - attr_Attr)
# print (len(attr_Attr - customers_Attr), 'Attribute value(s) exclusively in attr:',
#        '\n', attr_Attr - customers_Attr)
# print ('In', len(attr_Attr), 'features of attr,',
#        len(attr_Attr) - len(attr_Attr - customers_Attr), 'features are in Attribute of customers')

In [ ]:
# list(feature_dict.keys())[0]
# feature_dict.values()

In [ ]:
# values = np.array([1, 2, 3, 4])

# subtractions = np.subtract.outer(values, values)[np.triu_indices(len(values), k=1)]

# for result in subtractions:
#     print(result)

In [ ]:
# for i, (k, v) in enumerate(feature_dict.items()):
#     for j in range(i + 1, 4):
#             result = values[i] - values[j]
#             print(f"{values[i]} - {values[j]} = {result}")
#     print (i, k, v)
    
# for i, (k, v) in enumerate(zip(list(feature_dict.keys()), list(feature_dict.values()))):
#     print (i, (k, v))

In [ ]:
# '''
# customers에는 없고 attr에만 있는 42개 Attribute는 Desc가 없으으로
# (1.5 alignment) desc가 null이 아닌 행만 keep
# '''

# print ('customers에는 없고 attr에만 있는 42개 Attribute 수:', 
#        attr_num[attr_num.Desc.isna() == True].Attribute.nunique(),
#        '\n')

# attr_num = attr_num[attr_num.Desc.isna() == False]
# print (attr_num.info())

In [ ]:
# '''
# str data of column Value
# '''
# # attr_num with only numeric values in Value
# attr_str = attr[attr['Value_dtype'] == str].copy()
# print (attr_str.shape) 

# # add Desc and Information level
# attr_str = vlookup(attr_str, feature_desc, 'Attribute', ['Desc', 'Additional notes'])
# print (attr_str.info())
# attr_str.head()

In [ ]:
# '''
# 3개 null attribute는 정수와 문자를 모두 값을로 갖는 것들로 42개는 유효하고
# 이 42개는 다음 section에서 볼 예정이므로 (1.5 alignment) desc가 null이 아닌 행만 keep
# '''

# # attr_str[attr_str.Desc.isna() == True] # int와 str을 모두 갖는 셀. 따라서 42는 맞음.... 이 별로 중요하지도 않은 것을 남겨야 하나...

# attr_str = attr_str[attr_str.Desc.isna() == False]
# print (attr_str.info())

In [ ]:
# # attr_str_to_check = pv_attr_str[pv_attr_str.Meaning.str.contains('numeric value')].Attribute
# attr_str_to_check = pv_attr_str[
#     pv_attr_str.index.get_level_values(2).str.contains('numeric value')].index.get_level_values(0)
# # 먼저 추후 체크할 것들을 뽑아 놓고
# attr_str_to_check

In [ ]:
# pv_attr_str = pd.DataFrame(pv_attr_str[
#     ~(pv_attr_str.index.get_level_values(0).isin(attr_str_to_check))
#     &~(pv_attr_str.index.get_level_values(2) == 'unknown')
#     ].to_records())

# pv_attr_str = pd.pivot_table(
#     pv_attr_str,
#     index = ['Attribute', 'Desc'],
#     values = 'Value',
#     aggfunc = lambda x: list(x)
#     )

# view_all(pv_attr_str)

In [ ]:
# print(not_in_attr_str.values.T.shape)
# not_in_attr_str.values.T.tolist()

In [ ]:
# for i in concat_cont_ft[1:]:

#     min_val = cc_num[i].min()
#     max_val = cc_num[i].max()
#     bin_interval = 1
#     bin_edges = np.arange(min_val, max_val + bin_interval, bin_interval)

#     desc_val = concat_num[concat_num['Attribute'] == i]['Desc'].values[0]
#     count_val = int(concat_num[concat_num['Attribute'] == i].fillna(0)['count'].values[0])

#     ax = cc_num[i].plot(
#         kind = 'hist',
#         figsize=(10, 1.5),
#         color='gray',
#         bins = bin_edges,
#         align = 'mid',
#         title = ('histogram - ' + desc_val + ' ' + i)
#         );
#     ax.set_xlabel('Values: Min: ' + str(int(min_val)) + ', Max: ' + str(int(max_val)));
#     plt.show()
    
#     ax = cc_num[i].plot(
#         kind = 'hist',
#         figsize=(10, 1.5),
#         color='gray',
#         bins = np.arange(-0.5, 11.5, 1),
#         align = 'mid',
#         title = ('histogram - ' + desc_val + ' - Value 0 to 10')
#         );
#     ax.set_xlabel('Values');
#     plt.show()
    
#     # Define the outlier thresholds by applying multiplier 5.0
#     q1 = concat_num[concat_num['Attribute'] == i]['25%'].values[0]
#     q3 = concat_num[concat_num['Attribute'] == i]['75%'].values[0]
#     iqr = q3 - q1
#     lower_threshold = q1 - 5.0 * iqr
#     upper_threshold = q3 + 5.0 * iqr

#     # Identify outliers
#     col_val = cc_num[i].values
#     outliers = sorted(set([i for i in col_val if i < lower_threshold or i > upper_threshold]), reverse = True)
#     for j in outliers:
#         print (int(j), '{:.1%}'.format(((cc_num[i] == j).sum())/count_val*100), end = ' ')
#     print ('\n', '==========' * 10)

In [ ]:
# for i in concat_cont_ft[1:]:

#     min_val = cc_num[i].min()
#     max_val = cc_num[i].max()
#     # bin_interval = 1
#     bin_edges = np.arange(min_val, max_val + 10, 10)

#     desc_val = concat_num[concat_num['Attribute'] == i]['Desc'].values[0]
#     count_val = int(concat_num[concat_num['Attribute'] == i].fillna(0)['count'].values[0])

#     ax = cc_num[i].plot(
#         kind = 'hist',
#         figsize=(10, 1.5),
#         color='gray',
#         bins = bin_edges,
#         align = 'mid',
#         title = ('histogram - ' + desc_val + ' ' + i)
#         );
#     ax.set_xlabel('Values: Min: ' + str(int(min_val)) + ', Max: ' + str(int(max_val)));
#     plt.show()
    
#     ax = cc_num[i].plot(
#         kind = 'hist',
#         figsize=(10, 1.5),
#         color='gray',
#         bins = np.arange(-0.5, 11.5, 1),
#         align = 'mid',
#         title = ('histogram - ' + desc_val + ' - Value 0 to 10')
#         );
#     ax.set_xlabel('Values');
#     plt.show()
    
#     # Define the outlier thresholds by applying multiplier 1.5
#     q1 = concat_num[concat_num['Attribute'] == i]['25%'].values[0]
#     q3 = concat_num[concat_num['Attribute'] == i]['75%'].values[0]
#     iqr = q3 - q1
#     lower_threshold = q1 - 1.5 * iqr
#     upper_threshold = q3 + 1.5 * iqr

#     # Identify outliers
#     col_val = cc_num[i].values
#     outliers = sorted(
#         set([i for i in col_val if i < lower_threshold or i > upper_threshold]),
#         reverse = True)
#     # for j in outliers:
#     #     print (int(j), '{:.1%}'.format(((cc_num[i] == j).sum())/count_val*100), end = ' ')
        
#     outlier_list = [str(int(j)) + ': ' + '{:.1%}'.format((cc_num[i] == j).sum() / count_val)
#                     for j in outliers]
    
#     print('Outliers (Value: %)')
#     for j in range(0, len(outlier_list), 10):
#         print (', '.join(outlier_list[j: j+10]))
#     print ('\n')

**[function]** score_meaning_val

In [ ]:
# def score_meaning_val(data, pv_idx = ['Attribute', 'Description', 'Desc', 'Additional notes']):
#     '''
#     function to check unique values of Score and Meaning by Attribute
#     data : dataframe to examine
#     pv_idx: list of pivot_table index
#     '''
#     data = vlookup(data, feature_desc, 'Attribute', ['Desc', 'Additional notes'], nan_val = 'no_info')
#     pv = pd.pivot_table(
#         data,
#         index = pv_idx,
#         values = ['Meaning', 'Score'],
#         aggfunc = lambda x: list(x))
    
#     return pv

In [ ]:
# def verify_null(data, null_list, pv_idx = ['Attribute', 'Description', 'Desc', 'Additional notes']):
#     '''
#     function to display the summary of Score and Meaning values,
#     when Meaning values are in the list containing possibly null values.   
    
#     data: dataframe to examine
#     null_list: list of possibly null values
#     pv_idx: pivot_table index
#     '''
#     null_check_Attribute = data[data['Meaning'].isin(null_list)]['Attribute'].to_list()
#     null_check = data[data['Attribute'].isin(null_check_Attribute)]
    
#     null_check = vlookup(null_check, feature_desc, 'Attribute', ['Desc', 'Additional notes'], fill_na = 'no_info')
#     pv = pd.pivot_table(
#         null_check,
#         index = pv_idx,
#         values = ['Score', 'Meaning'],
#         aggfunc = lambda x: list(x))
    
#     return pv

In [ ]:
# # For values of Attribute without Description, add 12 Description values from attr
# attr_to_add = attr[['Attribute', 'Description']].copy()
# attr_to_add.dropna(inplace = True)
# attr_to_add = attr_to_add.rename(columns = {'Description': 'Description_to_add'})

# feature_desc = vlookup(feature_desc, attr_to_add, 'Attribute')
# feature_desc.Description = np.where(
#     ((feature_desc.Description.isnull() == True) & (feature_desc.Description_to_add.isnull() == False)),
#     feature_desc.Description_to_add,
#     feature_desc.Description)
# feature_desc = feature_desc.drop('Description_to_add', axis=1)

# # For values of Attribute without Description, make colum of translation (ger_to_eng)
# def ger_to_eng (ger_text):
#     '''
#     function to translate German text
#     '''    
#     translator = Translator(service_urls=['translate.google.com'])    
#     try:
#         translation = translator.translate(ger_text, src='de', dest='en')
#         return translation.text        
#     except:
#         return np.nan

# feature_desc['ger_to_eng'] = np.where(
#     feature_desc.Description.isnull(),
#     feature_desc.Attribute.str.replace('_', ' ').apply(ger_to_eng),
#     np.nan)
# feature_desc['Desc'] = feature_desc.Description.fillna('') + feature_desc.ger_to_eng.fillna('')

# feature_desc.info()

In [ ]:
# # pv_min_max 2.3.1 변경 for merge
# min_max_copy = pd.DataFrame(pv_min_max.to_records())
# min_max_copy.columns = list(
#     min_max_copy.columns[:2]) + list(eval(i)[0] for i in min_max_copy.columns[2:])
# min_max_copy

In [ ]:
# import pandas as pd
# import numpy as np
# from IPython.display import display
# import sys
# from contextlib import ExitStack

# def view_all (data):
#     '''
#     function to display all contents of data
#     '''
#     # with ExitStack() as stack:
#     #     stack.enter_context(pd.option_context(
#     #     'display.max_rows', None,
#     #     'display.max_colwidth', None,
#     #     'display.max_seq_items', None
#     #     ))
#     #     stack.enter_context(np.printoptions(threshold=np.inf))
    
#     # np.set_printoptions(threshold = sys.maxsize)
    
#     with pd.option_context(
#     'display.max_rows', None, 
#     'display.max_colwidth', None,
#     'display.max_seq_items', None,
#     ):
#         display(data)

In [ ]:
# '''
# categorize pre-processing items
# '''
# # # categorize p_process_items
# # p_process_all = p_process_items.query('by_feature.str.contains("All")')
# # p_process_items = p_process_items.query('not by_feature.str.contains("All")')

# # p_process_list = ['p_process_all']
# p_process_list = []
# for i in p_process_items.method.unique():
#     globals()['p_process_' + i] = p_process_items.query('method == "{}"'.format(i))
#     globals()['p_process_' + i].name = 'p_process_' + i
#     p_process_list.append('p_process_' + i)

# p_process_list

In [ ]:
# correlation_matrix = customers.corr(numeric_only = True)
# correlation_matrix

### eod